# Parte 2 — Modelado y Evaluación

**Parte 2 — Modelado y Evaluación**

### 🎯 **Objetivo**
Entrenar un modelo de regresión para predecir la variable `price_usd`.

### 🔹 **Lineamientos**
*   **Modelo:** Pueden elegir el que consideren más adecuado (ej. **Linear Regression, RandomForest, XGBoost, LightGBM**, etc.).
*   **División de datos:** Separar los datos en conjuntos de **entrenamiento (train) y prueba (test)**. El porcentaje de división es libre, pero debe estar justificado.
*   **Proceso iterativo:** Para llegar al modelo final, se deben construir varios modelos intermedios. Es crucial explicar las conclusiones parciales obtenidas en cada iteración y por qué se eligió el modelo final.

### 🔹 **Métricas de Evaluación**
Utilizar al menos una de las siguientes métricas:
*   **RMSE** (Root Mean Squared Error)
*   **MAE** (Mean Absolute Error)
*   **R²:** Pueden utilizarlo como guía para evaluar el ajuste del modelo.

> 💡 **Extra:** Comparar RMSE y MAE puede ayudar a entender cómo los outliers están afectando el rendimiento del modelo.

# Librerías a importar

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import OneHotEncoder
import os

import time
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
#!pip install funpymodeling

In [ ]:
from funpymodeling.exploratory import status, freq_tbl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importo base generada en la parte 1

In [ ]:
properati =pd.read_csv('/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/dataset/processed.csv', sep = ',')

In [ ]:
properati.shape

(104512, 9)

In [ ]:
status(properati)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,price_usd,0,0.0,0,0.0,4311,float64
1,surface_total,0,0.0,0,0.0,374,float64
2,surface_covered,0,0.0,0,0.0,182,float64
3,rooms,0,0.0,0,0.0,5,float64
4,bedrooms,0,0.0,0,0.0,5,float64
5,bathrooms,0,0.0,0,0.0,3,float64
6,property_type,0,0.0,0,0.0,10,object
7,state_name,0,0.0,0,0.0,4,object
8,place_name,0,0.0,0,0.0,84,object


In [ ]:
properati.head(5)

,price_usd,surface_total,surface_covered,rooms,bedrooms,bathrooms,property_type,state_name,place_name
0,259000.0,62.0,55.0,3.0,2.0,2.0,Departamento,Capital Federal,Colegiales
1,235500.0,77.0,67.0,3.0,2.0,2.0,Departamento,Capital Federal,Almagro
2,175000.0,60.0,55.0,2.0,2.0,1.0,Departamento,Capital Federal,Villa Urquiza
3,140000.0,74.0,47.0,2.0,1.0,1.0,PH,Capital Federal,Boedo
4,173000.0,66.0,64.0,3.0,2.0,1.0,Departamento,Capital Federal,Almagro


Variables Categóricas:
1.   property_type
2.   country_name
3.   state_name
4.   place_name

In [ ]:
print(properati['property_type'].unique())
print(properati['property_type'].nunique())

['Departamento' 'PH' 'Oficina' 'Casa' 'Local comercial' 'Depósito' 'Lote'
 'Otro' 'Cochera' 'Casa de campo']
10


In [ ]:
print(properati['state_name'].unique())
print(properati['state_name'].nunique())

['Capital Federal' 'Bs.As. G.B.A. Zona Norte' 'Bs.As. G.B.A. Zona Sur'
 'Bs.As. G.B.A. Zona Oeste']
4


In [ ]:
#verifico frecuencia de place_name
print(properati['place_name'].nunique())
freq_tbl(properati['place_name'])


84


,place_name,frequency,percentage,cumulative_perc
0,Palermo,6286,0.060146,0.060146
1,Quilmes,6249,0.059792,0.119938
2,Lomas de Zamora,5751,0.055027,0.174966
3,Morón,4714,0.045105,0.220070
4,Belgrano,4699,0.044961,0.265032
...,...,...,...,...
79,Esteban Echeverría,13,0.000124,0.999751
80,Catalinas,10,0.000096,0.999847
81,Florencio Varela,7,0.000067,0.999914
82,Sin Información,6,0.000057,0.999971


In [ ]:
#creo una copia
properati_encoded = properati.copy()

# Modelos

In [ ]:
# Crear encoder
property_type_ohe = OneHotEncoder(sparse_output=False, drop='first')

# Ajustar y transformar
property_type_encoded = property_type_ohe.fit_transform(properati_encoded[['property_type']])

# Crear DataFrame con nombres de columnas
property_type_columns = [f"property_type_{cat}" for cat in property_type_ohe.categories_[0][1:]]
property_type_encoded_df = pd.DataFrame(
    property_type_encoded,
    columns=property_type_columns,
    index=properati_encoded.index
)

# Concatenar con el dataset
properati_encoded = pd.concat([
    properati_encoded.drop(columns=['property_type']),
    property_type_encoded_df
], axis=1)


In [ ]:
# Crear encoder
state_name_ohe = OneHotEncoder(sparse_output=False, drop='first')

# Ajustar y transformar
state_encoded = state_name_ohe.fit_transform(properati_encoded[['state_name']])

# Crear DataFrame con nombres de columnas
state_columns = [f"state_name_{cat}" for cat in state_name_ohe.categories_[0][1:]]
state_encoded_df = pd.DataFrame(
    state_encoded,
    columns=state_columns,
    index=properati_encoded.index
)

# Concatenar con el dataset
properati_encoded = pd.concat([
    properati_encoded.drop(columns=['state_name']),
    state_encoded_df
], axis=1)


In [ ]:
print(properati_encoded.head())
print("\nColumnas actuales:", properati_encoded.columns.tolist())


   price_usd  surface_total  surface_covered  rooms  bedrooms  bathrooms  \
0   259000.0           62.0             55.0    3.0       2.0        2.0   
1   235500.0           77.0             67.0    3.0       2.0        2.0   
2   175000.0           60.0             55.0    2.0       2.0        1.0   
3   140000.0           74.0             47.0    2.0       1.0        1.0   
4   173000.0           66.0             64.0    3.0       2.0        1.0   

      place_name  property_type_Casa de campo  property_type_Cochera  \
0     Colegiales                          0.0                    0.0   
1        Almagro                          0.0                    0.0   
2  Villa Urquiza                          0.0                    0.0   
3          Boedo                          0.0                    0.0   
4        Almagro                          0.0                    0.0   

   property_type_Departamento  property_type_Depósito  \
0                         1.0                     0.0

In [ ]:
# place_name muchas categorías distintas (alta cardinalidad)
#Frequency Encoding: reeemplaza cada categoría por la proporción de veces que aparece en el dataset.

place_name_freq = properati_encoded['place_name'].value_counts(normalize=True)
properati_encoded['place_name_freq'] = properati_encoded['place_name'].map(place_name_freq)
properati_encoded = properati_encoded.drop(columns=['place_name'])

print(properati_encoded[['place_name_freq']].head())


   place_name_freq
0         0.011166
1         0.024284
2         0.024925
3         0.006640
4         0.024284


In [ ]:
print(properati_encoded.shape)
print("Columnas finales:", properati_encoded.columns.tolist())
print(properati_encoded.head())

(104512, 19)
Columnas finales: ['price_usd', 'surface_total', 'surface_covered', 'rooms', 'bedrooms', 'bathrooms', 'property_type_Casa de campo', 'property_type_Cochera', 'property_type_Departamento', 'property_type_Depósito', 'property_type_Local comercial', 'property_type_Lote', 'property_type_Oficina', 'property_type_Otro', 'property_type_PH', 'state_name_Bs.As. G.B.A. Zona Oeste', 'state_name_Bs.As. G.B.A. Zona Sur', 'state_name_Capital Federal', 'place_name_freq']
   price_usd  surface_total  surface_covered  rooms  bedrooms  bathrooms  \
0   259000.0           62.0             55.0    3.0       2.0        2.0   
1   235500.0           77.0             67.0    3.0       2.0        2.0   
2   175000.0           60.0             55.0    2.0       2.0        1.0   
3   140000.0           74.0             47.0    2.0       1.0        1.0   
4   173000.0           66.0             64.0    3.0       2.0        1.0   

   property_type_Casa de campo  property_type_Cochera  \
0           

In [ ]:
# Guardar los archivos
os.makedirs('/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model', exist_ok=True)

In [ ]:
# Guardar preprocesadores con pickle
with open('/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/place_name_freq.pkl', 'wb') as f:
    pickle.dump(place_name_freq, f)

with open('/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/property_type_ohe', 'wb') as f:
    pickle.dump(property_type_ohe, f)

with open('/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/state_name_ohe.pkl', 'wb') as f:
    pickle.dump(state_name_ohe, f)

# Guardar lista de columnas features - excluyo price_usd (columna target)
columnas_features = list(properati_encoded.drop(columns=['price_usd']).columns)
with open('/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/columnas_features.pkl', 'wb') as f:
    pickle.dump(columnas_features, f)

print("Columnas finales:", properati_encoded.columns.tolist())

Columnas finales: ['price_usd', 'surface_total', 'surface_covered', 'rooms', 'bedrooms', 'bathrooms', 'property_type_Casa de campo', 'property_type_Cochera', 'property_type_Departamento', 'property_type_Depósito', 'property_type_Local comercial', 'property_type_Lote', 'property_type_Oficina', 'property_type_Otro', 'property_type_PH', 'state_name_Bs.As. G.B.A. Zona Oeste', 'state_name_Bs.As. G.B.A. Zona Sur', 'state_name_Capital Federal', 'place_name_freq']


División de datos: Separar los datos en conjuntos de entrenamiento (train) y prueba (test).

In [ ]:
X = properati_encoded.drop('price_usd', axis=1)
y = properati_encoded['price_usd']

el conjunto de entrenamiento necesita suficiente información para aprender patrones, y el test debe ser representativo para evaluar generalización. La relación 0.3 / 0.7 es lo habitual en dataset de estos tamaños

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
#import time
#import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
#from sklearn.linear_model import LinearRegression
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
results = []

# --- Iteración 1: Regresión Lineal ---
start = time.time()
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
end = time.time()

results.append({
    'Modelo': 'Regresión Lineal',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_lr)),
    'MAE': mean_absolute_error(y_test, y_pred_lr),
    'R²': r2_score(y_test, y_pred_lr),
    'Tiempo (segundos)': round(end - start, 2)
})

# --- Iteración 2: Random Forest (default) ---
start = time.time()
rf_default = RandomForestRegressor(random_state=42)
rf_default.fit(X_train, y_train)
y_pred_rf_default = rf_default.predict(X_test)
end = time.time()

results.append({
    'Modelo': 'Random Forest (default)',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf_default)),
    'MAE': mean_absolute_error(y_test, y_pred_rf_default),
    'R²': r2_score(y_test, y_pred_rf_default),
    'Tiempo (segundos)': round(end - start, 2)
})

# --- Iteración 3: Random Forest (más árboles) ---
start = time.time()
rf_500 = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
rf_500.fit(X_train, y_train)
y_pred_rf_500 = rf_500.predict(X_test)
end = time.time()

results.append({
    'Modelo': 'Random Forest (500 árboles)',
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_rf_500)),
    'MAE': mean_absolute_error(y_test, y_pred_rf_500),
    'R²': r2_score(y_test, y_pred_rf_500),
    'Tiempo (segundos)': round(end - start, 2)
})

# --- Iteración 4: tres configuraciones optimizadas ---
param_combinations = [
    {'nombre': 'RF optimizado A (conservador)', 'n_estimators': 200, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 3},
    {'nombre': 'RF optimizado B (balanceado)',  'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 2},
    {'nombre': 'RF optimizado C (potente)',     'n_estimators': 500, 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 1},
]

for params in param_combinations:
    start = time.time()
    rf_model = RandomForestRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    end = time.time()

    results.append({
        'Modelo': params['nombre'],
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R²': r2_score(y_test, y_pred),
        'Tiempo (segundos)': round(end - start, 2)
    })

    # Guardar cada modelo en su variable correspondiente
    if params['nombre'] == 'RF optimizado A (conservador)':
        rf_A = rf_model
    elif params['nombre'] == 'RF optimizado B (balanceado)':
        rf_B = rf_model
    elif params['nombre'] == 'RF optimizado C (potente)':
        rf_C = rf_model
modelos = {
    "Regresión Lineal": lr,
    "RF Default": rf_default,
    "RF 500 árboles": rf_500,
    "RF optimizado A (conservador)": rf_A,
    "RF optimizado B (balanceado)": rf_B,
    "RF optimizado C (potente)": rf_C
}

# --- Mostrar resultados finales ---
results_df = pd.DataFrame(results).sort_values(by='RMSE')
print("\n📊 Resultados comparativos (ordenados por RMSE):")
print(results_df)



📊 Resultados comparativos (ordenados por RMSE):
                          Modelo          RMSE           MAE        R²  \
5      RF optimizado C (potente)  59211.732196  39685.600314  0.685204   
2    Random Forest (500 árboles)  59382.566699  39615.630271  0.683385   
1        Random Forest (default)  59488.568667  39689.740486  0.682254   
4   RF optimizado B (balanceado)  59600.721491  40954.742077  0.681054   
3  RF optimizado A (conservador)  61522.535192  42728.470447  0.660154   
0               Regresión Lineal  73016.808852  52250.961525  0.521305   

   Tiempo (segundos)  
5              47.49  
2             101.69  
1              25.09  
4              17.70  
3               9.65  
0               0.13  


In [ ]:

# --- Identificar el mejor modelo ---
best_model = results_df.iloc[0]
print(f"\n🏆 Mejor modelo: {best_model['Modelo']}")
print(f"   RMSE: {best_model['RMSE']:.2f}")
print(f"   MAE: {best_model['MAE']:.2f}")
print(f"   R²: {best_model['R²']:.3f}")
print(f"   Tiempo: {best_model['Tiempo (segundos)']} s")


🏆 Mejor modelo: RF optimizado C (potente)
   RMSE: 59211.73
   MAE: 39685.60
   R²: 0.685
   Tiempo: 47.49 s


# Guardado de archivos

In [ ]:
filename = '/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/rf_C.pkl'
pickle.dump(modelos["RF optimizado C (potente)"], open(filename, 'wb'))
#Es demasiado pesado

In [ ]:
from joblib import dump, load

filename = '/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/rf_C.joblib'
dump(modelos["RF optimizado C (potente)"], filename)
#sigue pesando lo mismo

['/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/rf_C.joblib']

In [ ]:
filename = '/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/rf_500.pkl'
pickle.dump(modelos["RF 500 árboles"], open(filename, 'wb'))
#pesa más

In [ ]:
filename = '/content/drive/MyDrive/BOOTCAMP - GERIES 4/TP Final/model/rf_default.pkl'
pickle.dump(modelos["RF Default"], open(filename, 'wb'))

# Justificación

**Objetivo del Modelado**
El objetivo es entrenar un modelo de regresión para predecir la variable price_usd de las propiedades inmobiliarias, basándonos en las características de las propiedades (como superficie, tipo de propiedad, y ubicación, entre otras).


**Variables incluidas:**

*surface_total y surface_covered:* las superficies de las propiedades tienen un gran imapctro a la hora de predecir su valor. Se incluyeron ambas debido a su fuerte correlación con el precio de las propiedades.

*property_type:* (departamento, casa, etc.) es de gran relevancia para la valoración, ya que cada tipo tiene un mercado y precio diferente.

*state_name y place_name:* la ubicación influye directamente en el valor de la propiedad. Se incluyeron tanto a nivel de estado / provincia como de barrio para capturar variaciones en el precio a nivel geográfico.

*Rooms, bedrooms y bathrooms* tienen un gran impacto en el precio de departamentos y casas.


**Variables descartadas:**

*Latitud y Longitud*, la ubicación geográfica ya se encuentra representada en las variables state_name y place_name, que indican la zona y barrio de la propiedad. Estas variables contienen información más directa y fácil de interpretar. Los inmuebles que están eu una misma coordenada de latitud y longitud pueden tener una gran variación de precios dependiendo de otros factores como la infraestructura de la zona. Además, estas variables podrían estar fuertemente correlacionadas entre sí ya que se refieren a una localización geográfica.

*Fecha de Publicación:* puede no tener una relación directa con el precio de la propiedad. El precio de venta es más sensible a factores como la ubicación, el tamaño y el tipo de propiedad, En particular, la información analizada fue en 2019 / 2020 y los precios de las propiedades se vieron afectados temporalmente por la pandemia que generó cambios en la demanda, ajustes de precio e incertidumbre económica.


**División de Datos (Train-Test Split)**
La división entre los conjuntos de entrenamiento y prueba:
•	70% para entrenamiento permite que el modelo tenga suficiente información para aprender patrones complejos.
•	30% para prueba garantiza que haya suficientes datos para evaluar el rendimiento del modelo en datos no vistos, lo que es clave para medir la capacidad de generalización del modelo.


**Modelos Analizados**
El modelo inicial elegido fue Regresión Lineal debido a su simplicidad.
Regresión Lineal: asume que la relación entre las variables independientes (como superficie, tipo de propiedad, etc.) y el precio de la propiedad es lineal.
Random Forest (modelo por defecto): los resultados mostraron mejoras en comparación con la regresión línea
Random Forest (más árboles): utiliza 500 árboles para mejorar el rendimiento,
Random Forest Optimizado: por úlitmo, se probaron tres configuraciones optimizadas de Random Forest con diferentes combinaciones de hiperparámetros.

**Métricas de Evaluación Utilizadas**

El RMSE es útil para evaluar modelos donde se desean predicciones lo más precisas posible.

El MAE mide el error promedio entre las predicciones y los valores reales. Es menos sensible a los outliers que el RMSE y proporciona una visión más equilibrada de la precisión del modelo.

El R² nos ayuda a entender qué tan bien se ajusta el modelo a los datos. Un valor de R² cercano a 1 indica que el modelo explica una gran parte de la variabilidad de los datos, mientras que valores cercanos a 0 indican que el modelo tiene poco poder explicativo.


**Justificación de la Elección del Modelo: Random Forest (default)**

El modelo mejor clasificado según las métricas fue RF optimizado C (potente), no obstante, no fue el modelo final.
Se eligió como modelo final al Random Forest (default) por varias razones, entre ellas el rendimiento y  la eficiencia computacional.

1. Rendimiento Aceptable con Menor Costo Computacional: aunque RF optimizado C y Random Forest con 500 árboles ofrecieron un mejor rendimiento, ambos modelos son más costosos computacionalmente. Random Forest (default) es un modelo con un rendimiento competitivo, lo que lo convierte en una excelente opción cuando el costo computacional es un factor crítico. Ofrece un buen equilibrio entre precisión y velocidad de entrenamiento.

2. Simplificación del Proceso: los modelos optimizados de Random Forest (como RF optimizado C y RF 500 árboles) requieren ajustes adicionales de parámetros como número de estimadores y profundidad máxima entre otros, lo que hace que el proceso de optimización sea más largo y dependiente de más pruebas. Por otro lado, Random Forest (default) usa parámetros estándar que aún ofrecen un buen desempeño en este caso y su simplicidad facilita la implementación rápida sin la necesidad de ajustes exhaustivos.

3. Buen Rendimiento Sin Necesidad de Ajustes Avanzados: presenta un RMSE y MAE competitivos demostrando que el modelo puede predecir bien el precio de las propiedades sin los costos adicionales de optimización. Es  la fase inicial de modelado y se logra obtener resultados rápidos, antes de considerar modelos más complejos.

4. Comparación de Resultados: en el análisis comparativo de resultados, aunque RF optimizado C obtuvo el mejor rendimiento en términos de RMSE y R², la diferencia con Random Forest (default) no era lo suficientemente significativa como para justificar la complejidad adicional del modelo más pesado. El tiempo de entrenamiento y el tiempo de predicción fueron considerablemente más rápidos para Random Forest (default).


**Conclusión**
La elección de Random Forest (default) fue justificada por su buen rendimiento en comparación con modelos más complejos, así como por su eficiencia computacional. Este modelo proporciona un equilibrio adecuado entre precisión y costo computacional, lo que lo hace una opción ideal para entornos donde se requiere un modelo eficaz sin comprometer demasiado los recursos de cómputo.

RMSE = 59,488 dólares, este valor sugiere que hay algunos errores grandes ya que el RMSE penaliza los errores grandes de manera más severa.

MAE = 39,689 dólares: en promedio, el modelo se equivoca en 39,689 dólares por predicción, sin importar la magnitud de los errores.

Esto sugiere que hay algunos outliers en las predicciones. En el primer paso probé diversas maneras para reducir los outliers y la utilizada fue la que consideré mejor para el modelo sin reducir significativamente la base.

El modelo explica aproximadamente el 68 % de la variabilidad de los precios. Los errores absolutos siguen siendo significativos, por lo que las predicciones individuales pueden diferir bastante del valor real. No obstante, esto es normal en datos inmobiliarios, donde intervienen muchos factores difíciles de modelar. Hay que tener en cuenta que hay factores no incluidos en los datos, por ejemplo disponibilidad de amenities, antiguedad y estado del inmueble entre otros.
